In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as pl

In [3]:
train_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/train"
test_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/train"
val_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/val"

In [11]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory = train_path, target_size = (150,150),batch_size = 32)
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=val_path, target_size=(150,150), batch_size=32)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(150,150), batch_size=32, shuffle=False)

Found 1185 images belonging to 2 classes.
Found 1185 images belonging to 2 classes.
Found 1185 images belonging to 2 classes.


In [5]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                 

In [17]:
x = mobile.layers[-2].output
output = Dense(2, activation = "sigmoid")(x)
model = Model(inputs = mobile.input, outputs = output)

In [18]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics =["accuracy"])

In [19]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', restore_best_weights=True, verbose=1, patience=7)

In [ ]:
model.fit(x = train_batches,steps_per_epoch=len(train_batches),validation_data=val_batches, validation_steps=len(val_batches), epochs=100, callbacks=[es], verbose =1)

Epoch 1/100
13/38 [=========>....................] - ETA: 2:25 - loss: 0.2911 - accuracy: 0.9519